In [1]:
### '2016-01-04' ~ '2020-01-31'

In [21]:
SMAPE

pytorch_forecasting.metrics.point.SMAPE

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from pytorch_forecasting import Baseline
from pytorch_forecasting.metrics import SMAPE

import joblib
import pickle
import optuna

warnings.filterwarnings('ignore')
%matplotlib inline

/home/kyle/.pyenv/versions/3.8.3/envs/pytorch-forecasting/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kyle/.pyenv/versions/3.8.3/envs/pytorch-forecasting/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (SMAPE). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` 

In [2]:
torch.cuda.is_available()

True

### 함수정의

In [3]:
def sliding_windows(data, lookback_length, forecast_length):

    x = []
    y = []
    
    for i in range(lookback_length, len(data) - forecast_length + 1):
        _x = data[(i-lookback_length) : i]
        _y = data[i : (i + forecast_length)]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


def get_data_loader(X, y, batch_size):

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    train_dl = DataLoader(train_ds, batch_size = batch_size)

    val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    val_dl = DataLoader(val_ds, batch_size = batch_size)

    input_size = x_train.shape[-1]

    return train_dl, val_dl, input_size

### Data loading && Preproces

In [4]:
scaler = MinMaxScaler()

data=pd.read_excel('./data_full.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1858 non-null   datetime64[ns]
 1   Account DOW      1858 non-null   object        
 2   REV OBD          1858 non-null   int64         
 3   OBD NET+FSC_KRW  1858 non-null   float64       
 4   OBD A/R_KRW      1858 non-null   float64       
 5   REV CPN          1858 non-null   int64         
 6   CPN NET+FSC_KRW  1858 non-null   float64       
 7   CPN A/R_KRW      1858 non-null   float64       
 8   REV TKT          1858 non-null   int64         
 9   TKT NET+FSC_KRW  1858 non-null   float64       
 10  TKT A/R_KRW      1858 non-null   float64       
 11  WTI              1858 non-null   float64       
 12  exchanges        1858 non-null   float64       
 13  kospi            1858 non-null   float64       
 14  rates            1858 non-null   float64

In [5]:
res_data = data[data["Date"].isin(pd.date_range('2016-01-04', '2020-01-31'))]
res_data.reset_index(drop=True, inplace=True)
res_data = res_data[['Date', 'REV OBD']]
scale_cols = ['REV OBD']

In [6]:
res_data.shape

(1489, 2)

In [7]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

# Loockback_period & forecasting_period
max_prediction_length = 20
lookback_length = 100
training_data_max = len(res_data) - max_prediction_length

# 학습용 데이터
data_p = res_data.iloc[:training_data_max, :]
training_data = scaler.fit_transform(data_p[scale_cols])

In [8]:
training_data_max

1469

In [9]:
data_p.shape

(1469, 2)

In [10]:
data_p

,Date,REV OBD
0,2016-01-04,28000
1,2016-01-05,24657
2,2016-01-06,26920
3,2016-01-07,26624
4,2016-01-08,28879
...,...,...
1464,2020-01-07,24716
1465,2020-01-08,26562
1466,2020-01-09,26855
1467,2020-01-10,31204


#### Metric 생성을 위한 oot sample 정의 (원본 코드 LSTM-pytorch_optuna_05와 다른 부분)


In [12]:
# max_prediction_length 만큼의 데이터는 예측 데이터와 비교를 위해 분리
# Training set에 없는 데이터로 구성
# Input과 output의 pair로 정의
# Input과 output의 pair로 정의
from datetime import datetime, timedelta
pred_start = '2020-02-01'
pred_end = '2020-02-20'
pred_y_start = pred_start
pred_y_end = pred_end

pred_x_start = datetime.strftime(datetime.strptime(pred_start, '%Y-%m-%d') - timedelta( days = 60), '%Y-%m-%d')
pred_x_end =datetime.strftime(datetime.strptime(pred_start, '%Y-%m-%d') - timedelta( days = 1), '%Y-%m-%d')
print('pred_x_start = ', pred_x_start)
print('pred_x_end = ', pred_x_end)


x_for_metric = scaler.fit_transform(data[data["Date"].isin(pd.date_range(pred_x_start, pred_x_end))][scale_cols])
y_for_metric = scaler.fit_transform(data[data["Date"].isin(pd.date_range(pred_y_start, pred_y_end))][scale_cols])

# x_for_metric['time_index'] = np.arange(len(x_for_metric))
# x_for_metric['market'] = 'OBD'
print('x_for_metric', x_for_metric[:3])
print('y_for_metric', y_for_metric[:3])

pred_x_start =  2019-12-03
pred_x_end =  2020-01-31
x_for_metric [[0.15122259]
 [0.21618695]
 [0.38933502]]
y_for_metric [[1.        ]
 [0.78841891]
 [0.65602554]]


In [13]:
x_for_metric.shape

(60, 1)

In [14]:
y_for_metric.shape

(20, 1)

In [15]:
# LSTM은 1 step 뒤의 값만을 예측하므로, forecasting_period를 1로 두고 진행
x, y = sliding_windows(training_data, lookback_length, 1)

### Model 정의

In [16]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)
        
        self.fc = nn.Linear(hidden_size  * num_layers, num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size * self.num_layers)
        out = self.fc(h_out)
        return out

In [17]:
def train(log_interval, model, train_dl, val_dl, optimizer, criterion, epoch):

    best_loss = np.inf
    for epoch in range(epoch):
        train_loss = 0.0
        model.train()
        for data, target in train_dl:

            if False:
                data, target = data.cuda(), target.cuda()
                model = model.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target) # mean-squared error for regression
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        valid_loss = 0.0
        model.eval()
        for data, target in val_dl:

            if False:
                data, target = data.cuda(), target.cuda()

            output = model(data)         
            loss = criterion(output, target)
            valid_loss += loss.item()

        if ( epoch % log_interval == 0 ):
            print(f'\n Epoch {epoch} \t Training Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(val_dl)} \n')

        if best_loss > (valid_loss / len(val_dl)):
            print(f'Validation Loss Decreased({best_loss:.6f}--->{(valid_loss / len(val_dl)):.6f}) \t Saving The Model')
            best_loss = (valid_loss / len(val_dl))
            torch.save(model.state_dict(), 'lstm_saved_model.pth')

    return best_loss


In [25]:
def objective(trial):

    cfg = { 
            'batch_size' : trial.suggest_categorical('batch_size',[32, 64, 128, 256, 512]), # [64, 128, 256],
            # 'num_epochs' : trial.suggest_int('num_epochs', 5, 50, 5),
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1e-1), #trial.suggest_loguniform('learning_rate', 1e-2, 1e-1), # learning rate을 0.01-0.1까지 로그 uniform 분포로 사용
            'hidden_size': trial.suggest_categorical('hidden_size',[32, 64]), # , 128, 256, 512
            'num_layers': trial.suggest_int('num_layers', 1, 7, 1),       
        }

    torch.manual_seed(42)

    log_interval = 5
    num_classes = 1 # parameter에서 빼서 상수로 설정

    train_dl, val_dl, input_size = get_data_loader(x, y,  cfg['batch_size'])
    
    model = LSTM(
        num_classes = num_classes, 
        input_size = input_size, 
        hidden_size = cfg['hidden_size'], 
        num_layers = cfg['num_layers']
    )
    
    if False:
        model = model.cuda()
        
    optimizer = optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    criterion = torch.nn.MSELoss()
    best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, 5)

    print('best loss for the trial = ', best_loss)
    predict_data = []
    # 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
    x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

    for j, i in enumerate(range(max_prediction_length)):

        # feed the last forecast back to the model as an input
        x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
        xt_pred = torch.Tensor(x_pred)

        if False:
            xt_pred = xt_pred.cuda()
        # generate the next forecast
        yt_pred = model(xt_pred)
        # tensor to array
        # x_pred = xt_pred.cpu().detach().numpy()
        y_pred = yt_pred.cpu().detach().numpy()

        # save the forecast
        predict_data.append(y_pred)

    # transform the forecasts back to the original scale
    predict_data = np.array(predict_data).reshape(-1, 1)
    
    print('predicted = ', predict_data[-10:])
    print('actual = ', y_for_metric[-10:])

    print('SMAPE = ', SMAPE)
    _smape = np.mean(2*(abs(y_for_metric - predict_data) / (abs(y_for_metric) + abs(predict_data))))
    
    print(f' \nSMAPE : {_smape}')


    return _smape

In [26]:
sampler = optuna.samplers.TPESampler()
#   sampler = optuna.samplers.SkoptSampler()

# model.load_state_dict(torch.load('lstm_saved_model.pth'))
    
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials= 7)
joblib.dump(study, './LSTM_pytorch_optuna_06.pkl')

[I 2022-06-23 11:29:04,313] A new study created in memory with name: no-name-a26719ec-e38e-47d0-b370-64c38fcd258a



 Epoch 0 	 Training Loss: 0.04183424599468708 	 Validation Loss: 0.019451656999687355 

Validation Loss Decreased(inf--->0.019452) 	 Saving The Model
Validation Loss Decreased(0.019452--->0.017631) 	 Saving The Model
Validation Loss Decreased(0.017631--->0.017296) 	 Saving The Model
Validation Loss Decreased(0.017296--->0.017277) 	 Saving The Model
best loss for the trial =  0.017277449886831973


[W 2022-06-23 11:30:18,754] Trial 0 failed, because the value array([[0.98760382],
       [0.80135129],
       [0.65136044],
       [0.36555559],
       [0.35807498],
       [0.1808104 ],
       [0.55391763],
       [0.41867771],
       [0.32762872],
       [0.24202368],
       [0.64374948],
       [0.1858385 ],
       [0.48748516],
       [0.1657117 ],
       [0.13457899],
       [0.15139588],
       [0.32968391],
       [2.        ],
       [0.99917495],
       [1.85758069]]) could not be cast to float.


predicted =  [[0.3154639 ]
 [0.3167913 ]
 [0.31471592]
 [0.32010084]
 [0.32090485]
 [0.3182711 ]
 [0.3162536 ]
 [0.3086599 ]
 [0.31011367]
 [0.30744153]]
actual =  [[0.16183382]
 [0.26292454]
 [0.19136295]
 [0.37793739]
 [0.36720759]
 [0.27347699]
 [0.2267447 ]
 [0.        ]
 [0.10348497]
 [0.01135054]]
SMAPE =  <class 'pytorch_forecasting.metrics.point.SMAPE'>
 
SMAPE : [[0.98760382]
 [0.80135129]
 [0.65136044]
 [0.36555559]
 [0.35807498]
 [0.1808104 ]
 [0.55391763]
 [0.41867771]
 [0.32762872]
 [0.24202368]
 [0.64374948]
 [0.1858385 ]
 [0.48748516]
 [0.1657117 ]
 [0.13457899]
 [0.15139588]
 [0.32968391]
 [2.        ]
 [0.99917495]
 [1.85758069]]

 Epoch 0 	 Training Loss: 1.0761222293334347 	 Validation Loss: 0.05589917219347424 

Validation Loss Decreased(inf--->0.055899) 	 Saving The Model
Validation Loss Decreased(0.055899--->0.025690) 	 Saving The Model


In [ ]:
study = joblib.load('./LSTM_pytorch_optuna_06.pkl') 
df = study.trials_dataframe() 
df.sort_values('value')

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_hidden_size,params_learning_rate,params_num_epochs,params_num_layers,state
5,5,64.258700,2022-06-22 11:35:30.269737,2022-06-22 11:35:43.250624,0 days 00:00:12.980887,32,32,0.052172,25,4,COMPLETE
0,0,65.264598,2022-06-22 11:33:58.832678,2022-06-22 11:34:19.662926,0 days 00:00:20.830248,128,128,0.003809,45,2,COMPLETE
3,3,66.497066,2022-06-22 11:35:16.333515,2022-06-22 11:35:18.986343,0 days 00:00:02.652828,512,32,0.025897,50,3,COMPLETE
2,2,67.335145,2022-06-22 11:35:10.961908,2022-06-22 11:35:16.331743,0 days 00:00:05.369835,256,256,0.006021,45,1,COMPLETE
1,1,71.487864,2022-06-22 11:34:19.666222,2022-06-22 11:35:10.960080,0 days 00:00:51.293858,512,512,0.066099,50,5,COMPLETE
4,4,80.762639,2022-06-22 11:35:18.988171,2022-06-22 11:35:30.267761,0 days 00:00:11.279590,64,32,0.012539,25,6,COMPLETE
6,6,80.906707,2022-06-22 11:35:43.254234,2022-06-22 11:36:40.391254,0 days 00:00:57.137020,64,512,0.005986,35,5,COMPLETE


In [ ]:
study.best_params

{'batch_size': 32,
 'hidden_size': 32,
 'learning_rate': 0.05217185390739443,
 'num_epochs': 25,
 'num_layers': 4}

In [ ]:
# 실제 예측하려는 target 구간의 max_encoder_length : 100일 
test_data = res_data[res_data["Date"].isin(pd.date_range('2019-10-24', '2020-01-31'))]
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1389 to 1488
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     100 non-null    datetime64[ns]
 1   REV OBD  100 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.3 KB


In [ ]:
test_data.shape

(100, 2)

In [ ]:
torch.manual_seed(42)

log_interval = 5
num_classes = 1 # parameter에서 빼서 상수로 설정

train_dl, val_dl, input_size = get_data_loader(x, y,  study.best_params['batch_size'])

model = LSTM(
    num_classes = num_classes, 
    input_size = input_size, 
    hidden_size = study.best_params['hidden_size'], 
    num_layers = study.best_params['num_layers']
)

if torch.cuda.is_available():
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=study.best_params['learning_rate'])
criterion = torch.nn.MSELoss()
best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, study.best_params['num_epochs'])

print('best loss for the trial = ', best_loss)
predict_data = []
# 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

for j, i in enumerate(range(max_prediction_length)):

    # feed the last forecast back to the model as an input
    x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
    xt_pred = torch.Tensor(x_pred)

    if torch.cuda.is_available():
        xt_pred = xt_pred.cuda()
    # generate the next forecast
    yt_pred = model(xt_pred)
    # tensor to array
    # x_pred = xt_pred.cpu().detach().numpy()
    y_pred = yt_pred.cpu().detach().numpy()

    # save the forecast
    predict_data.append(y_pred)

# transform the forecasts back to the original scale
predict_data = np.array(predict_data).reshape(-1, 1)
print('predicted = ', predict_data[-10:])
print('actual = ', y_for_metric[-10:])
_smape = SMAPE()(y_for_metric, predict_data)

print(f' \nSMAPE : {_smape}')

NameError: name 'torch' is not defined

In [ ]:
# actual_data는 max_prediction_length에 해당하는 20일의 기간 : 실제 예측하려는 기간 (2020년 2월1일 ~ 2020년 2월 20일)
actual_data = data[data["Date"].isin(pd.date_range('2020-02-01', '2020-02-20'))]
actual_data['time_index'] = np.arange(len(actual_data))
actual_data['market'] = 'OBD'
actual_data.head()

,Date,Account DOW,REV OBD,OBD NET+FSC_KRW,OBD A/R_KRW,REV CPN,CPN NET+FSC_KRW,CPN A/R_KRW,REV TKT,TKT NET+FSC_KRW,TKT A/R_KRW,WTI,exchanges,kospi,rates,stock_a,stock_k,stock_kkj,time_index,market
1492,2020-02-01,SAT,23210.0,1.172142e+10,505015.9678,6412.0,2.781090e+09,433732.0616,3888.0,3.517716e+09,9.047624e+05,51.56,1194.5,2119.01,1.30,21763.0,15335.0,43860.0,0,OBD
1493,2020-02-02,SUN,20824.0,9.549017e+09,458558.2667,6049.0,2.553490e+09,422134.2552,3675.0,2.836992e+09,7.719705e+05,51.56,1194.5,2119.01,1.30,21763.0,15335.0,43860.0,1,OBD
1494,2020-02-03,MON,19331.0,8.695239e+09,449808.0166,17749.0,1.013166e+10,570829.9001,10027.0,1.117746e+10,1.114736e+06,50.11,1194.0,2118.88,1.29,21883.0,15659.0,44739.0,2,OBD
1495,2020-02-04,TUE,17277.0,7.375559e+09,426900.4457,16530.0,9.081702e+09,549407.2659,9351.0,1.003142e+10,1.072765e+06,49.61,1188.5,2157.90,1.33,22790.0,16697.0,47035.0,3,OBD
1496,2020-02-05,WED,17211.0,7.884009e+09,458079.6780,18526.0,1.005432e+10,542713.8500,10438.0,1.124609e+10,1.077418e+06,50.75,1184.5,2165.63,1.31,22909.0,16859.0,46937.0,4,OBD


In [ ]:
# 실제 예측하려는 target prediction 구간 : 20일
actuals = actual_data['REV OBD']
actuals

1492    23210.0
1493    20824.0
1494    19331.0
1495    17277.0
1496    17211.0
1497    16299.0
1498    18470.0
1499    17570.0
1500    17037.0
1501    16587.0
1502    13758.0
1503    14898.0
1504    14091.0
1505    16195.0
1506    16074.0
1507    15017.0
1508    14490.0
1509    11933.0
1510    13100.0
1511    12061.0
Name: REV OBD, dtype: float64

In [ ]:
actuals = torch.tensor(actual_data['REV OBD'].values)
print(actuals)
print(actuals.shape)

tensor([23210., 20824., 19331., 17277., 17211., 16299., 18470., 17570., 17037.,
        16587., 13758., 14898., 14091., 16195., 16074., 15017., 14490., 11933.,
        13100., 12061.], dtype=torch.float64)
torch.Size([20])


In [ ]:
baseline_predictions =  torch.tensor([data.loc[data.Date == '2020-01-31']['REV OBD'].values[0]] * len(actuals)).to(torch.float32)

mae_with_baseline = (actuals - baseline_predictions).abs().mean().item()
smape_with_baseline = (2 * (baseline_predictions - actuals).abs() / (baseline_predictions.abs() + actuals.abs() + 1e-8)).mean()

print(f'mae_with_baseline : {mae_with_baseline}')
print(f'smape_with_baseline : {smape_with_baseline}')

mae_with_baseline : 7632.35
smape_with_baseline : 0.39083401983455535


In [ ]:
torch.tensor([data.loc[data.Date == '2020-01-31']['REV OBD'].values[0]] * len(actuals)).to(torch.float32)

tensor([23904., 23904., 23904., 23904., 23904., 23904., 23904., 23904., 23904.,
        23904., 23904., 23904., 23904., 23904., 23904., 23904., 23904., 23904.,
        23904., 23904.])

In [ ]:
predicts = scaler.inverse_transform(predict_data)
print(predicts.shape)
print(predicts)
print(type(predicts))

(20, 1)
[[25458.75 ]
 [25387.09 ]
 [25091.88 ]
 [25435.121]
 [25151.662]
 [25639.234]
 [25292.557]
 [25087.676]
 [24852.074]
 [24863.977]
 [25256.951]
 [25193.934]
 [24965.936]
 [24552.059]
 [25093.479]
 [25416.496]
 [25019.54 ]
 [24737.062]
 [24675.943]
 [24816.951]]
<class 'numpy.ndarray'>


In [ ]:
predicts = torch.tensor(predicts)
predicts.shape

torch.Size([20, 1])

In [ ]:
mae_with_test = (actuals - predicts).abs().mean().item()
smape_with_test = SMAPE()(actuals.numpy(), predicts.numpy())

print(f'mae_with_test : {mae_with_test}')
print(f'smape_with_test : {smape_with_test}')

mae_with_test : 8827.7685546875
smape_with_test : 874.3154316016535


In [ ]:
y_train.shape

np.expand_dims(pred, axis=1).shape

predicted_for_plot = sc.inverse_transform(np.concatenate((y_train, np.expand_dims(pred, axis=1))))
actual_for_plot = sc.inverse_transform(np.concatenate((y_train, y_test)))

predicted_for_plot.shape

actual_for_plot.shape

plt.plot(predicted_for_plot[300:])
plt.plot(actual_for_plot[300:])
plt.suptitle('Timeseries Prediction')
plt.axvline(x=y_train.shape[0]-300, c='r', linestyle='--')
plt.show()

NameError: ignored